In [1]:
from selenium import webdriver
import os
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
from datetime import datetime
import db_multiproc_calc_neutral2 as dmc
from bs4 import BeautifulSoup
import requests
from random import *
import glob
import barchart_lib as bl

k_do_download = False

cwd = os.getcwd()

executable_path = os.path.join(cwd,'chromedriver.exe')
executable_path = os.path.join(cwd,'chromedriver')
display(executable_path)

date_value = datetime.now()
#date_value = pd.to_datetime('03-31-2023')
date_str = date_value.strftime("%Y-%m-%d")

#date_str = '2023-03-24'
output_date_str = datetime.now().strftime("%Y%m%d")
file_date_str = date_str
barchart_datestr = pd.to_datetime(file_date_str).strftime('%m-%d-%Y')

download_path = os.path.join(cwd, 'local_download', file_date_str)

print(f'processing futures options:{file_date_str}')

try:
    os.mkdir(download_path)
except:
    print(f'{download_path} exists')


'/home/gitrepo/futureDataCapture/chromedriver'

processing futures options:2023-04-05
/home/gitrepo/futureDataCapture/local_download/2023-04-05 exists


In [2]:
def get_nearest_contract(date_value):
    fut_con = pd.read_csv(os.path.join(cwd, 'local_download', 'futures_contract_dates.csv'), parse_dates=['first_notice','expiration'])
    # extract the symbol root which is the first two characters of the contract
    fut_con['symbol_root'] = fut_con['futures_contract'].map(lambda x:x[0:2])
    # get the contracts which are still valid
    idx_valid = fut_con['first_notice'] > date_value
    fut_con_valid = fut_con.loc[idx_valid]
    # extract the nearest
    df_ret = fut_con_valid.sort_values(['first_notice'], ascending=True).groupby('symbol_root').head(1).sort_values('futures_contract')
    return df_ret

def find_all_downloaded_contracts(download_path):
    df_all_ctr = find_all_downloaded_contracts(download_path)
    return df_all_ctr['futures_contract'].unique()

def find_all_downloaded_details(download_path):
    all_df = []
    for filename in glob.glob(os.path.join(download_path, '*.txt')):
        all_df.append(pd.read_csv(filename))
    df_all_ctr = pd.concat(all_df, axis=0)
    return df_all_ctr

def get_treasury_data(start_date):

    start_date_str = start_date.strftime("%Y%m")
    url='https://home.treasury.gov/resource-center/data-chart-center/interest-rates/pages/xml?data=daily_treasury_yield_curve&field_tdr_date_value_month='+start_date_str
    soup = BeautifulSoup(requests.get(url).text,'lxml')
    table = soup.find_all('m:properties')
    tbondvalues = []
    for i in table:
        tbondvalues.append([i.find('d:new_date').text[:10],i.find('d:bc_1month').text,i.find('d:bc_2month').text,i.find('d:bc_3month').text,i.find('d:bc_6month').text,i.find('d:bc_1year').text,i.find('d:bc_2year').text,i.find('d:bc_3year').text,i.find('d:bc_5year').text,i.find('d:bc_10year').text,i.find('d:bc_20year').text,i.find('d:bc_30year').text])
    ustcurve = pd.DataFrame(tbondvalues,columns=['date','1m','2m','3m','6m','1y','2y','3y','5y','10y','20y','30y'])
    ustcurve.iloc[:,1:] = ustcurve.iloc[:,1:].apply(pd.to_numeric)/100
    ustcurve['date'] = pd.to_datetime(ustcurve['date'])
    tbl_daily = ustcurve.sort_values(['date'])
    tbl_daily = tbl_daily.set_index('date').stack(level=0).reset_index()
    tbl_daily.columns = ['date', 'tenor', 'rate']

    return tbl_daily

def get_put_call_ratio(df_iv):
    # This takes the IV datafrmae and calcs both the delta adjusted put call and
    # natural put call ratio from the barcharts data
    df_iv = df_iv.dropna().copy()
    df_iv = df_iv.loc[:, ['strike', 'call_put', 'open_interest', 'delta']]
    df_iv['Delta-Adj-Put-Call-Ratio'] = df_iv['open_interest'] * df_iv['delta']
    df_iv = df_iv.drop(columns=['delta'])
    df_iv = df_iv.set_index(['strike', 'call_put']).groupby('call_put').sum()
    df_iv = df_iv.rename(columns={'open_interest':'Put-Call-Ratio'})
    pc_temp = df_iv.T
    return (pc_temp.loc[:,'P'] / pc_temp.loc[:,'C'])


def read_and_clean(file_name, parse_dates=[]):
    bar_chart_column_map = {
        'Prev Open':'Open',
        'Prev High':'High',
        'Prev Low':'Low',
        'Open Int':'Open Interest',
        'Prev Volume':'Volume',
        'Prev Change':'Change'
    }
    df_out = pd.read_csv(file_name.lower(), parse_dates=parse_dates, header=0, skipfooter=1, engine='python')
    df_out = df_out.rename(columns=bar_chart_column_map)
    return df_out

def get_iv_data_set(sym, option_expiration, file_date_str, barchart_datestr):
    # compile the implied vol dataframe from the files downloaded from barcharts

    exp_date_str = option_expiration
    exp_file_date_str = pd.to_datetime(exp_date_str).strftime('%m_%d_%y')

    #iv_str = df_exp_data.values[0][1]
    #all_iv = pd.to_numeric(iv_str.replace('%',''))/100

    px_data_path = os.path.join('local_download', file_date_str, f'{sym}_daily_historical-data-{barchart_datestr}.csv')
    opx_data_path = os.path.join('local_download', file_date_str, f'{sym}-options-american-options-exp-{exp_file_date_str}-show-all-%futuresoptionsview%-daily-{barchart_datestr}.csv')
    gr_data_path = os.path.join('local_download', file_date_str, f'{sym}-volatility-greeks-exp-{exp_file_date_str}-show-all-{barchart_datestr}.csv')

    # read the options price data
    #fpx = pd.read_csv(px_data_path.lower(), parse_dates=['Time']).dropna()
    fpx = read_and_clean(px_data_path, parse_dates=['Time']).dropna()
    fpx['date'] = fpx['Time']
    fpx = fpx.drop(columns=['Time'])
    fpx=fpx.set_index(['date']).sort_index()
    close_px = fpx.loc[barchart_datestr, :]['Last']

    # read in the option prices and transform
    #opx = pd.read_csv(opx_data_path.lower())
    opx = read_and_clean(opx_data_path)
    opx = opx.loc[:, ['Last', 'Strike', 'Volume', 'Open Interest']]
    opx['call_put']=opx['Strike'].map(lambda x:x[-1])
    opx['num_strike']=opx['Strike'].map(lambda x:pd.to_numeric(x[:-1].replace('-','.').replace(',','')))
    opx = opx.drop(columns=['Strike'])
#    if 'Low' in opx.columns:
#        opx=opx.drop(labels=['Strike', 'Delta', 'Open', 'High', 'Low', 'Change', 'Premium'], axis=1)
#        if 'Exp Date' in opx.columns:
#            opx=opx.drop(labels=['Exp Date'], axis=1)
#        opx = opx.rename(columns={'num_strike': 'Strike',
#                              'Open Interest':'open_interest',
#                              'Last':'mean_price'})
#    else:
#        opx=opx.drop(labels=['Strike', 'High', 'Low', 'Change', 'Premium'], axis=1)
    opx = opx.rename(columns={'num_strike': 'Strike',
                          'Open Interest':'open_interest',
                          'Last':'mean_price'})
    opx = opx.set_index(['Strike', 'call_put'])

    # read in the greeks and transform
    gr = read_and_clean(gr_data_path)

    gr['symbol']=sym
    gr['option_expiration']=pd.to_datetime(exp_date_str)
    gr['Strike']=gr['Strike'].map(lambda x:pd.to_numeric(x))
    gr['IV']=pd.to_numeric(gr['IV'].str[:-1])/100
    #RWMgr['IV']=all_iv
    gr['Type'] = gr['Type'].map({'Put':'P', 'Call':'C'})
    gr=gr.drop(labels=['IV Skew', 'Time', 'Last'], axis=1)
    gr=gr.rename(columns={'Type':'call_put'})
    gr=gr.set_index(['Strike', 'call_put'])

    # concat and make columns lowercase
    t=pd.concat([opx, gr], axis=1)
    t['close_px'] = close_px
    t = t.reset_index()
    t.columns = map(str.lower, t.columns)
    return t

def clean_implied_vol(df_iv_file):
    x = df_iv_file.set_index(['strike', 'call_put']).sort_index().copy()
    y = x['iv'].unstack('call_put')
    idx_put_zero = y['P']==0
    idx_call_zero = y['C']==0

    idx_call_zero_replace = idx_call_zero * ~idx_put_zero
    idx_put_zero_replace = idx_put_zero * ~idx_call_zero
    y.loc[idx_call_zero_replace, 'C'] = y.loc[idx_call_zero_replace, 'P']
    y.loc[idx_put_zero_replace, 'P'] = y.loc[idx_put_zero_replace, 'C']
    y = y.rolling(3, center=True, min_periods=0).median()
    y = y.stack('call_put')
    y.name = 'clean_iv'
    x1 = pd.concat([x,y],axis=1)
    x1['iv'] = x1['clean_iv']
    x1.drop(columns='clean_iv', inplace=True)
    return x1

def generate_neutral_futures(sym, option_expiration, file_date_str, barchart_datestr):
    t = get_iv_data_set(sym, option_expiration, file_date_str, barchart_datestr)
    df_iv = clean_implied_vol(t).dropna().reset_index()

    query_date = barchart_datestr
    root_symbol = sym

    unique_strikes = sorted(t['strike'])
    spot_prices = np.array(dmc.calc_spot_price_levels(unique_strikes), dtype=float)

    agg_neutral, df_greeks_by_strike, exp_neutral = \
        dmc.calc_daily_neutral_values( \
            df_iv,
            query_date,
            rf_rate,
            root_symbol,
            spot_prices)

    agg_neutral['Close'] = t['close_px'].values[0]
    agg_neutral['Symbol'] = sym
    return [agg_neutral, df_greeks_by_strike, exp_neutral]

In [3]:
rate_filename = os.path.join(download_path,'rate.csv')

df_rate = get_treasury_data(pd.to_datetime(date_str))
df_rate.to_csv(rate_filename, index=False)

df_rate = pd.read_csv(rate_filename)
rf_rate = df_rate.set_index(['date', 'tenor']).loc[date_str,'1m'].values[0]
rf_rate

0.0462

In [4]:
df_contract_list = get_nearest_contract(date_value)

In [5]:
all_sym = df_contract_list['futures_contract'].values
all_sym

array(['CLK23', 'GCM23', 'HGK23', 'NGK23', 'SIK23', 'ZCK23', 'ZNM23',
       'ZSK23', 'ZWK23'], dtype=object)

In [6]:
# use to debug a single symbol
#all_done = ['GCJ23', 'CLK23']
#all_sym = np.setdiff1d(all_sym, all_done)
#all_sym = ['GCJ23']

In [7]:
if k_do_download:
    driver = bl.bar_chart_get_driver(executable_path, download_path)
    bl.bar_chart_login(driver)
    for sym in all_sym:
        print(f'download {sym}')
        bl.bar_chart_download(driver, date_str, sym, download_path, k_do_download)
    driver.quit()

In [8]:
all_sym = find_all_downloaded_details(download_path)
all_contracts = all_sym[['futures_contract', 'option_expiration', 'iv']].drop_duplicates()
all_contracts

,futures_contract,option_expiration,iv
0,GCM23,04/25/23,17.52%
1,GCM23,05/25/23,17.52%
0,ZSK23,04/21/23,13.29%
1,ZSN23,05/26/23,16.58%
0,ZCK23,04/21/23,17.99%
1,ZCN23,05/26/23,21.50%
0,SIK23,04/25/23,32.86%
1,SIN23,05/25/23,32.24%
0,ZNM23,04/21/23,8.61%
1,ZNM23,05/26/23,8.61%


In [9]:
temp = []
for idx, row in all_contracts.iterrows():
    sym = row[0]
    option_expiration = row[1]
    print(row)
    temp.append(generate_neutral_futures(sym, option_expiration, file_date_str, barchart_datestr))

futures_contract        GCM23
option_expiration    04/25/23
iv                     17.52%
Name: 0, dtype: object
futures_contract        GCM23
option_expiration    05/25/23
iv                     17.52%
Name: 1, dtype: object
futures_contract        ZSK23
option_expiration    04/21/23
iv                     13.29%
Name: 0, dtype: object
futures_contract        ZSN23
option_expiration    05/26/23
iv                     16.58%
Name: 1, dtype: object
futures_contract        ZCK23
option_expiration    04/21/23
iv                     17.99%
Name: 0, dtype: object
futures_contract        ZCN23
option_expiration    05/26/23
iv                     21.50%
Name: 1, dtype: object
futures_contract        SIK23
option_expiration    04/25/23
iv                     32.86%
Name: 0, dtype: object
futures_contract        SIN23
option_expiration    05/25/23
iv                     32.24%
Name: 1, dtype: object
futures_contract        ZNM23
option_expiration    04/21/23
iv                      8.61%
Name: 

In [10]:
# Pull the data together and get the delta put call ratio from the files
temp_iv_data = []
for idx, row in all_contracts.iterrows():
    sym = row[0]
    option_expiration = row[1]

    df_iv = get_iv_data_set(sym, option_expiration, file_date_str, barchart_datestr)
    df_pcr = get_put_call_ratio(df_iv)
    df_iv['Put-Call-Ratio'] = df_pcr['Put-Call-Ratio']
    df_iv['Delta-Adj-Put-Call-Ratio'] = np.abs(df_pcr['Delta-Adj-Put-Call-Ratio'])
    df_iv = df_iv[['symbol', 'option_expiration', 'close_px', 'Put-Call-Ratio', 'Delta-Adj-Put-Call-Ratio']].tail(1)
    temp_iv_data.append(df_iv)
closepx_table = pd.concat(temp_iv_data).set_index('symbol')
closepx_table = closepx_table.rename(columns={'close_px':'Futures Close', 'option_expiration':'Opex Date'})

In [11]:
# Get the delta neutral and gamma neutral values and combine with final table
expn = [x[0] for x in temp]
final_output = pd.concat(expn).loc[-1,['symbol', 'delta_neutral', 'gamma_neutral' ]].set_index('symbol')
final_output = pd.concat([closepx_table, final_output], axis=1)
final_output['Last Trade Date'] = date_str
final_output = final_output.loc[:,['Last Trade Date', 'Opex Date', 'Futures Close', 'delta_neutral', 'gamma_neutral', 'Put-Call-Ratio', 'Delta-Adj-Put-Call-Ratio']]
final_output

,Last Trade Date,Opex Date,Futures Close,delta_neutral,gamma_neutral,Put-Call-Ratio,Delta-Adj-Put-Call-Ratio
symbol,,,,,,,
GCM23,2023-04-05,2023-04-25,2035.600000,1930.009883,1933.950509,0.955575,0.086512
GCM23,2023-04-05,2023-05-25,2035.600000,1873.580385,1854.898058,0.650671,0.106574
ZSK23,2023-04-05,2023-04-21,1511.000000,1506.367829,1523.854712,1.181500,0.950115
ZSN23,2023-04-05,2023-05-26,1477.750000,1437.122110,1399.422074,0.462541,0.480248
ZCK23,2023-04-05,2023-04-21,652.750000,645.825026,644.554112,1.001277,0.734883
ZCN23,2023-04-05,2023-05-26,627.500000,624.983456,628.504563,1.030316,1.056306
SIK23,2023-04-05,2023-04-25,25.037000,22.551017,22.300220,0.968958,0.089592
SIN23,2023-04-05,2023-05-25,25.248000,22.657366,22.678491,1.023535,0.159551
ZNM23,2023-04-05,2023-04-21,116.578125,114.151415,114.698946,0.844765,0.196336


In [12]:
final_output.to_csv(os.path.join(download_path,f'{output_date_str}.futures.neutrals.csv'), float_format='%2.2f')

# Testing Follows

In [13]:
final_output_target = pd.read_csv(os.path.join(download_path,f'finalout.target.{file_date_str}.csv'), parse_dates=['Opex Date']).set_index('symbol')
test_target_sym = ['GCJ23', 'CLK23', 'NGJ23']

FileNotFoundError: [Errno 2] No such file or directory: '/home/gitrepo/futureDataCapture/local_download/2023-04-05/finalout.target.2023-04-05.csv'

In [ ]:
final_output.loc[test_target_sym,:]

In [ ]:
 final_output_target.loc[test_target_sym, :]

In [ ]:
# Check output against test
all_columns = final_output.columns

for c in all_columns:
    t1 = final_output.loc[test_target_sym,c]
    t2 = final_output_target.loc[test_target_sym,c]
    try:
        pd.testing.assert_series_equal(t1, t2, rtol=0.01, check_exact=False)
    except:
        display(pd.concat([t1, t2], axis=1))


In [ ]:
all_df = []
for filename in glob.glob(os.path.join(download_path, '*.csv')):
    all_df.append(filename)

In [ ]:
all_df_new = all_df

In [ ]:
all_df_new = [x.replace('04-01-2023', '03-31-2023') for x in all_df_new]

In [ ]:
for oldname, newname in zip(all_df, all_df_new):
    os.rename(oldname, newname)